# 成年人死亡率预测

<br>
<hr>

## 1. 实验介绍

### 1.1 实验背景

成年人死亡率指的是每一千人中15岁至60岁死亡的概率（数学期望）。这里我们给出了世界卫生组织（WHO）下属的全球卫生观察站（GHO）数据存储库跟踪的所有国家健康状况以及许多其他相关因素。要求利用训练数据建立回归模型，并预测成年人死亡率（**Adult Mortality**）。

### 1.2 实验要求

1. 训练数据包含`2336`条记录和`22`个字段，对训练数据进行一定的可视化数据分析（章节2.2）
2. 利用训练数据，选择合适的信息作为特征建立回归模型，并预测测试数据成年人死亡率
3. 利用`MO`平台进行模型性能评估

### 1.3 实验环境
可以使用基于 Python 的 `Pandas` 库进行数据相关处理，使用 `Sklearn` 库进行相关模型构建。


### 1.4 注意事项
1. 推荐使用基于 Python 的`Sklearn`库进行相关实验
2. 数据中可能会有一些字段的值存在缺失

## 2. 实验部分

### 2.1 导入相关包

In [1]:
import pandas as pd
import sklearn
import numpy as np

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn import ensemble

import joblib

### 2.2 数据读取和可视化分析

In [2]:
# 读取数据集
train_data = pd.read_csv('./data/train_data.csv')

In [3]:
train_data

,Country,Year,Status,Life expectancy,infant deaths,Alcohol,percentage expenditure,Hepatitis B,Measles,BMI,...,Total expenditure,Diphtheria,HIV/AIDS,GDP,Population,thinness 1-19 years,thinness 5-9 years,Income composition of resources,Schooling,Adult Mortality
0,Seychelles,2000,Developing,71.8,0,8.24,601.760812,98.0,0,27.1,...,4.62,98.0,0.1,7578.851530,81131.0,6.3,6.7,0.000,12.3,188
1,Swaziland,2000,Developing,48.4,3,7.19,25.216833,83.0,10,25.9,...,5.26,84.0,46.4,1637.456700,161468.0,9.4,9.6,0.516,9.4,536
2,Togo,2000,Developing,54.6,14,1.10,2.029644,NaN,3578,16.6,...,4.35,64.0,5.1,26.393293,497367.0,9.4,9.3,0.425,9.3,339
3,United States of America,2000,Developed,76.8,28,8.21,0.000000,9.0,85,6.1,...,13.70,94.0,0.1,NaN,NaN,0.8,0.7,NaN,NaN,114
4,Panama,2000,Developing,75.7,2,5.58,9.871021,NaN,0,45.9,...,7.76,98.0,0.2,46.299350,33347.0,2.4,2.3,0.716,12.1,121
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2331,Micronesia (Federated States of),2015,Developing,69.4,0,NaN,0.000000,78.0,0,69.4,...,NaN,72.0,0.1,NaN,NaN,0.2,0.2,0.637,11.7,166
2332,Brazil,2015,Developing,75.0,42,NaN,0.000000,96.0,214,56.1,...,NaN,96.0,0.1,8757.262200,2596218.0,2.7,2.6,0.754,15.2,142
2333,Morocco,2015,Developing,74.3,17,NaN,0.000000,99.0,17,58.5,...,NaN,99.0,0.1,2847.285569,3483322.0,6.4,6.2,0.645,12.1,95
2334,Greece,2015,Developing,81.0,0,NaN,0.000000,96.0,1,66.5,...,NaN,99.0,0.1,187.789910,182883.0,0.8,0.7,0.865,17.2,72


可以看到训练数据包含2336条记录和22个字段，每个字段含义说明如下：
1. Country：国家
2. Year：年份
3. Status：发达国家或发展中国家
4. Life expectancy：预期寿命
5. Infant deaths：每千人口中的婴儿死亡人数
6. Alcohol：人均酒精消费量（以升纯酒精为单位）
7. percentage expenditure：卫生支出占人均国内生产总值的百分比
8. Hepatitis B：一岁儿童乙型肝炎免疫疫苗接种率
9. Measles：麻疹每1000人报告的病例数
10. BMI：所有人群平均BMI指数
11. under-five deaths：每千人口中五岁以下死亡人数
12. Polio：1岁儿童脊髓灰质炎免疫覆盖率（%）
13. Total expenditure：政府卫生支出占政府总支出的百分比
14. Diphtheria：1岁儿童白喉、破伤风类毒素和百日咳免疫接种率（%）
15. HIV/AIDS：每千名活产婴儿死于艾滋病毒/艾滋病（0-4岁）
16. GDP：人均国内生产总值（美元）
17. Population：人口
18. thinness 1-19 years：10至19岁儿童和青少年的消瘦流行率
19. thinness 5-9 years：5至9岁儿童中的消瘦流行率
20. Income composition of resources：财力收入构成方面的人类发展指数（从0到1）
21. Schooling：受教育年限
22. Adult Mortality：成人死亡率（每1000人中15至60岁死亡的概率）

其中`Adult Mortality`字段为要预测的标签(`label`)

In [4]:
# 查看数据集大小
train_data.shape

(2336, 22)

In [5]:
train_data.head()

,Country,Year,Status,Life expectancy,infant deaths,Alcohol,percentage expenditure,Hepatitis B,Measles,BMI,...,Total expenditure,Diphtheria,HIV/AIDS,GDP,Population,thinness 1-19 years,thinness 5-9 years,Income composition of resources,Schooling,Adult Mortality
0,Seychelles,2000,Developing,71.8,0,8.24,601.760812,98.0,0,27.1,...,4.62,98.0,0.1,7578.851530,81131.0,6.3,6.7,0.000,12.3,188
1,Swaziland,2000,Developing,48.4,3,7.19,25.216833,83.0,10,25.9,...,5.26,84.0,46.4,1637.456700,161468.0,9.4,9.6,0.516,9.4,536
2,Togo,2000,Developing,54.6,14,1.10,2.029644,NaN,3578,16.6,...,4.35,64.0,5.1,26.393293,497367.0,9.4,9.3,0.425,9.3,339
3,United States of America,2000,Developed,76.8,28,8.21,0.000000,9.0,85,6.1,...,13.70,94.0,0.1,NaN,NaN,0.8,0.7,NaN,NaN,114
4,Panama,2000,Developing,75.7,2,5.58,9.871021,NaN,0,45.9,...,7.76,98.0,0.2,46.299350,33347.0,2.4,2.3,0.716,12.1,121


In [6]:
# 计算各个特征之间的皮尔森相关系数
train_data.corr()

,Year,Life expectancy,infant deaths,Alcohol,percentage expenditure,Hepatitis B,Measles,BMI,under-five deaths,Polio,Total expenditure,Diphtheria,HIV/AIDS,GDP,Population,thinness 1-19 years,thinness 5-9 years,Income composition of resources,Schooling,Adult Mortality
Year,1.000000,0.173044,-0.045574,-0.059393,0.027351,0.119861,-0.089535,0.097642,-0.051892,0.099639,0.095594,0.146898,-0.144159,0.094351,0.012174,-0.049637,-0.055414,0.244558,0.219591,-0.085130
Life expectancy,0.173044,1.000000,-0.200307,0.403930,0.385335,0.250026,-0.156621,0.563239,-0.227572,0.470405,0.227194,0.500060,-0.558550,0.466012,-0.028874,-0.481919,-0.476773,0.720200,0.756970,-0.695485
infant deaths,-0.045574,-0.200307,1.000000,-0.117509,-0.088942,-0.233426,0.502258,-0.227356,0.996460,-0.168905,-0.127455,-0.186393,0.027106,-0.112767,0.559472,0.462870,0.466544,-0.148067,-0.197907,0.078613
Alcohol,-0.059393,0.403930,-0.117509,1.000000,0.351033,0.076217,-0.045272,0.328996,-0.114326,0.223054,0.297324,0.225653,-0.053149,0.367267,-0.032890,-0.437330,-0.425520,0.451557,0.545457,-0.192501
percentage expenditure,0.027351,0.385335,-0.088942,0.351033,1.000000,0.020959,-0.056564,0.243372,-0.091140,0.153229,0.196985,0.149584,-0.102281,0.908855,-0.030910,-0.257604,-0.258956,0.386001,0.399577,-0.244770
Hepatitis B,0.119861,0.250026,-0.233426,0.076217,0.020959,1.000000,-0.122937,0.166267,-0.243389,0.470815,0.054130,0.610947,-0.112097,0.084776,-0.125556,-0.128664,-0.130000,0.196330,0.214588,-0.161392
Measles,-0.089535,-0.156621,0.502258,-0.045272,-0.056564,-0.122937,1.000000,-0.176633,0.510569,-0.143172,-0.106506,-0.151227,0.033809,-0.078137,0.248298,0.224253,0.217004,-0.133857,-0.142477,0.024766
BMI,0.097642,0.563239,-0.227356,0.328996,0.243372,0.166267,-0.176633,1.000000,-0.237654,0.287353,0.231999,0.304195,-0.243694,0.318139,-0.074910,-0.531130,-0.535272,0.507016,0.562572,-0.378400
under-five deaths,-0.051892,-0.227572,0.996460,-0.114326,-0.091140,-0.243389,0.510569,-0.237654,1.000000,-0.187704,-0.128945,-0.208636,0.040501,-0.116458,0.545763,0.463884,0.467471,-0.167404,-0.214304,0.093789
Polio,0.099639,0.470405,-0.168905,0.223054,0.153229,0.470815,-0.143172,0.287353,-0.187704,1.000000,0.133347,0.666200,-0.165835,0.219403,-0.036908,-0.237836,-0.237604,0.380321,0.419043,-0.289902


In [7]:
# 将相关性矩阵绘制成热力图
corr = train_data.corr()
corr.style.background_gradient(cmap='coolwarm')

,Year,Life expectancy,infant deaths,Alcohol,percentage expenditure,Hepatitis B,Measles,BMI,under-five deaths,Polio,Total expenditure,Diphtheria,HIV/AIDS,GDP,Population,thinness 1-19 years,thinness 5-9 years,Income composition of resources,Schooling,Adult Mortality
Year,1.000000,0.173044,-0.045574,-0.059393,0.027351,0.119861,-0.089535,0.097642,-0.051892,0.099639,0.095594,0.146898,-0.144159,0.094351,0.012174,-0.049637,-0.055414,0.244558,0.219591,-0.085130
Life expectancy,0.173044,1.000000,-0.200307,0.403930,0.385335,0.250026,-0.156621,0.563239,-0.227572,0.470405,0.227194,0.500060,-0.558550,0.466012,-0.028874,-0.481919,-0.476773,0.720200,0.756970,-0.695485
infant deaths,-0.045574,-0.200307,1.000000,-0.117509,-0.088942,-0.233426,0.502258,-0.227356,0.996460,-0.168905,-0.127455,-0.186393,0.027106,-0.112767,0.559472,0.462870,0.466544,-0.148067,-0.197907,0.078613
Alcohol,-0.059393,0.403930,-0.117509,1.000000,0.351033,0.076217,-0.045272,0.328996,-0.114326,0.223054,0.297324,0.225653,-0.053149,0.367267,-0.032890,-0.437330,-0.425520,0.451557,0.545457,-0.192501
percentage expenditure,0.027351,0.385335,-0.088942,0.351033,1.000000,0.020959,-0.056564,0.243372,-0.091140,0.153229,0.196985,0.149584,-0.102281,0.908855,-0.030910,-0.257604,-0.258956,0.386001,0.399577,-0.244770
Hepatitis B,0.119861,0.250026,-0.233426,0.076217,0.020959,1.000000,-0.122937,0.166267,-0.243389,0.470815,0.054130,0.610947,-0.112097,0.084776,-0.125556,-0.128664,-0.130000,0.196330,0.214588,-0.161392
Measles,-0.089535,-0.156621,0.502258,-0.045272,-0.056564,-0.122937,1.000000,-0.176633,0.510569,-0.143172,-0.106506,-0.151227,0.033809,-0.078137,0.248298,0.224253,0.217004,-0.133857,-0.142477,0.024766
BMI,0.097642,0.563239,-0.227356,0.328996,0.243372,0.166267,-0.176633,1.000000,-0.237654,0.287353,0.231999,0.304195,-0.243694,0.318139,-0.074910,-0.531130,-0.535272,0.507016,0.562572,-0.378400
under-five deaths,-0.051892,-0.227572,0.996460,-0.114326,-0.091140,-0.243389,0.510569,-0.237654,1.000000,-0.187704,-0.128945,-0.208636,0.040501,-0.116458,0.545763,0.463884,0.467471,-0.167404,-0.214304,0.093789
Polio,0.099639,0.470405,-0.168905,0.223054,0.153229,0.470815,-0.143172,0.287353,-0.187704,1.000000,0.133347,0.666200,-0.165835,0.219403,-0.036908,-0.237836,-0.237604,0.380321,0.419043,-0.289902


In [8]:
# 查看相关矩阵
corr_matrix = train_data.corr()
corr_matrix["Adult Mortality"].sort_values(ascending = False)

Adult Mortality                    1.000000
 HIV/AIDS                          0.512732
 thinness 5-9 years                0.300346
 thinness  1-19 years              0.295657
under-five deaths                  0.093789
infant deaths                      0.078613
Measles                            0.024766
Population                        -0.001169
Year                              -0.085130
Total expenditure                 -0.118360
Hepatitis B                       -0.161392
Alcohol                           -0.192501
percentage expenditure            -0.244770
Polio                             -0.289902
GDP                               -0.296923
Diphtheria                        -0.304850
 BMI                              -0.378400
Income composition of resources   -0.452480
Schooling                         -0.462488
Life expectancy                   -0.695485
Name: Adult Mortality, dtype: float64

In [ ]:
# 利用seaborn检查可视化数据之间的依赖关系
import seaborn as sns
corr.style.background_gradient(cmap='coolwarm')
sns.pairplot(train_data)

/usr/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


### 2.3 模型拟合和成年人死亡率预测

要求同学们利用训练数据来建立回归模型，我们已经给出了一个可运行的基础代码，同学们可以对此进行参考

In [9]:
train_data = pd.read_csv('./data/train_data.csv')
model_filename = './model.pkl'
imputer_filename = './imputer.pkl'
scaler_filename = './scaler.pkl'

In [10]:
def preprocess_data(data, imputer=None, scaler=None):
    
    column_name = ['Year', 'Life expectancy ', 'infant deaths', 'Alcohol',
               'percentage expenditure', 'Hepatitis B', 'Measles ', ' BMI ', 'under-five deaths ',
               'Polio', 'Total expenditure', 'Diphtheria ', ' HIV/AIDS', 'GDP', 'Population',
               ' thinness  1-19 years', ' thinness 5-9 years', 'Income composition of resources',
               'Schooling']
    data = data.drop(["Country", "Status"], axis=1)
    
    if imputer==None:
        imputer = SimpleImputer(strategy='mean', missing_values=np.nan)
        imputer = imputer.fit(data[column_name])
    data[column_name] = imputer.transform(data[column_name])
    
    if scaler==None:
        scaler = MinMaxScaler()
        scaler = scaler.fit(data)
    data_norm = pd.DataFrame(scaler.transform(data), columns=data.columns)
    
    data_norm = data_norm.drop(['Year'], axis = 1)
    
    return data_norm, imputer, scaler

In [11]:
def model_fit(train_data):
    
    train_y = train_data.iloc[:,-1].values
    train_data = train_data.drop(["Adult Mortality"], axis=1)
    train_data_norm, imputer, scaler = preprocess_data(train_data)
    
    train_x = train_data_norm.values
    
    regressor = ensemble.RandomForestRegressor(n_estimators = 2022)
    regressor.fit(train_x, train_y)
    
    joblib.dump(regressor, model_filename)
    joblib.dump(imputer, imputer_filename)
    joblib.dump(scaler, scaler_filename)
    
    return regressor

In [12]:
def predict(test_data, filename):
    loaded_model = joblib.load(model_filename)
    imputer = joblib.load(imputer_filename)
    scaler = joblib.load(scaler_filename)

    test_data_norm, _, _ = preprocess_data(test_data, imputer, scaler)
    test_x = test_data_norm.values
    predictions = loaded_model.predict(test_x)
    
    return predictions


In [13]:
model = model_fit(train_data)

In [14]:
# 打印模型的截距
print(model.intercept_)

AttributeError: 'RandomForestRegressor' object has no attribute 'intercept_'

In [15]:
# 打印模型的斜率
print(model.coef_)

AttributeError: 'RandomForestRegressor' object has no attribute 'coef_'

### 2.4 模型性能评估

我们可以看一下模型在训练集上的表现：

In [16]:
label = train_data.loc[:,'Adult Mortality']
data = train_data.iloc[:,:-1]
y_pred = predict(data, './model.pkl')
r2 = r2_score(label, y_pred)
mse = mean_squared_error(label, y_pred)
print("MSE is {}".format(mse))
print("R2 score is {}".format(r2))

MSE is 884.5058282182479
R2 score is 0.9431065378474165


## 3. 作业提交

**作业要求及注意事项**：    
             
1. 点击左侧栏提交作业后点击【生成文件】则需要勾选与预测 predict() 函数的 cell相关的其它cell ，并将其转化成为 main.py 文件。                       
2. 请导入必要的包和第三方库以及该模型所依赖的 py 文件 (包括此文件中曾经导入过的)。             
3. 请加载你认为训练最佳的模型，即请按要求填写模型路径。              
4. `predict()` 函数的输入输出及函数名称请不要改动。
5. 提交的作业包括【程序报告.pdf】和对应的ipynb文件

===========================================  **模型预测代码答题区域**  ===========================================  
在下方的代码块中编写 **模型预测** 部分的代码，请勿在别的位置作答

In [16]:
import pandas as pd
import sklearn
import numpy as np

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

import joblib

model_filename = './model.pkl'
imputer_filename = './imputer.pkl'
scaler_filename = './scaler.pkl'

def preprocess_data(data, imputer=None, scaler=None):
    
# -------------------------- 请调整你的数据预处理过程 ---------------------------
## 输入：
#### data 为 pandas.DataFrame类型数据
#### imputer 为缺失值填充方式
#### scaler 为数据归一化方式
## 输出：
#### data_norm 为处理后的数据，为 pandas.DataFrame类型数据
   
    column_name = ['Year', 'Life expectancy ', 'infant deaths', 'Alcohol',
               'percentage expenditure', 'Hepatitis B', 'Measles ', ' BMI ', 'under-five deaths ',
               'Polio', 'Total expenditure', 'Diphtheria ', ' HIV/AIDS', 'GDP', 'Population',
               ' thinness  1-19 years', ' thinness 5-9 years', 'Income composition of resources',
               'Schooling']
    data = data.drop(["Country", "Status"], axis=1)
    
    if imputer==None:
        imputer = SimpleImputer(strategy='mean', missing_values=np.nan)
        imputer = imputer.fit(data[column_name])
    data[column_name] = imputer.transform(data[column_name])
    
    if scaler==None:
        scaler = MinMaxScaler()
        scaler = scaler.fit(data)
    data_norm = pd.DataFrame(scaler.transform(data), columns=data.columns)
    
    data_norm = data_norm.drop(['Year'], axis = 1)
    
    return data_norm

def predict(test_data):
    
# -------------------------- 请加载您最满意的模型 ---------------------------
# 加载模型(请加载你认为的最佳模型)
# 加载模型,加载请注意 filename 是相对路径, 与当前文件同级。 
# test_data 为 pandas.DataFrame类型数据
    loaded_model = joblib.load(model_filename)
    imputer = joblib.load(imputer_filename)
    scaler = joblib.load(scaler_filename)
    
    test_data_norm = preprocess_data(test_data, imputer, scaler)
    test_x = test_data_norm.values
    predictions = loaded_model.predict(test_x)
    
    return predictions
